In [1]:
!pip install catboost

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [3]:
dataset = pd.read_csv("train.csv")
dataset.head()

,Year,Month,Consumer_type,Consumption,Consumer_number,Installation_zone
0,2013,1,domestic,0,MOGV36480546611521,Installation_zone 1
1,2013,1,industrial,5,BECS02817768252637,Installation_zone 2
2,2013,1,domestic,6,VRFW65577141436242,Installation_zone 2
3,2013,1,domestic,1,QLLI18662653137621,Installation_zone 2
4,2013,1,domestic,13,HYUO61823402850645,Installation_zone 2


In [4]:
le_iz = LabelEncoder()
le_iz.fit(dataset['Installation_zone'].unique())

le_ct = LabelEncoder()
le_ct.fit(dataset['Consumer_type'].unique())

le_cn = LabelEncoder()
le_cn.fit(dataset['Consumer_number'].unique())

dataset['Installation_zone'] = le_iz.transform(dataset['Installation_zone'])
dataset['Consumer_type'] = le_ct.transform(dataset['Consumer_type'])
dataset['Consumer_number'] = le_cn.transform(dataset['Consumer_number'])

In [5]:
X = dataset.drop('Consumer_type', axis='columns')
y = dataset['Consumer_type']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
cat_features = X.select_dtypes(include=['object']).columns.tolist()  # Convert to list

# Create a CatBoost Pool object for training and testing data
train_pool = Pool(X_train, label=y_train, cat_features=cat_features)
test_pool = Pool(X_test, label=y_test, cat_features=cat_features)

In [7]:
# Define CatBoost model
clf = CatBoostClassifier(
    iterations=100000,  # Number of boosting iterations
    learning_rate=0.1,  # Step size shrinkage used in update to prevent overfitting
    depth=6,  # Depth of the tree
    loss_function='MultiClass',  # Specify the loss function for multiclass classification
    random_seed=42
)

In [8]:
# Train the model
clf.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=10, verbose=False)

In [9]:
# Make predictions on the test set
y_pred = clf.predict(X_test)

In [10]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


Accuracy: 0.9154178346844458
